# Introduction

This tutorial demonstrates the most important basic steps involved in the analysis of scanning electron diffraction data.

The data was acquired from a GaAs nanowire adopting the zinc blende structure and exhibiting type I twinning (i.e. on {111}) along its length. The region of interest contains a single nanowire comprising multiple crystals each in one of the two twinned orientations and near to a <1-10> zone axis.

This functionaility has been checked to run in pyxem-0.9.0 (July 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

# Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#cal'> Alignment & Calibration</a>
3. <a href='#vdf'> Virtual Diffraction Imaging</a>
4. <a href='#ml'> Machine Learning SPED Data</a>
5. <a href='#vec'> Peak Finding & Vector Analysis</a>

Import pyXem and other required libraries

In [ ]:
%matplotlib tk
import pyxem as pxm
import numpy as np

Download and the data for this demo from here and put in directory with notebooks:

https://drive.google.com/drive/folders/1nkqDIu8g_kOQOuRqx5yDigtjopgv_Isj?usp=sharing

# <a id='loa'></a> 1. Loading and Inspection

Load the SPED data acquired from the nanowire

In [ ]:
dp = pxm.load_hspy('./nanowire_sed_example.hdf5',
                  assign_to='electron_diffraction2d')

Look at what kind of object 'dp' is

In [ ]:
dp.data.dtype

Inspect the metadata associated with the object 'dp'

In [ ]:
dp.metadata

Set important experimental parameters using the built in function

In [ ]:
dp.set_experimental_parameters(accelerating_voltage=300.,
                               camera_length=21.,
                               scan_rotation=277.,
                               convergence_angle=0.7,
                               exposure_time=10.)

See how this changed the metadata

In [ ]:
dp.metadata

Set another metadata item and check it

In [ ]:
dp.metadata.set_item("General.title", 'GaAs Nanowire')
dp.metadata

Plot the data to inspect it

In [ ]:
dp.plot(cmap='inferno', vmax=50)

# <a id='cal'></a> 2. Alignment & Calibration

Apply distortion corrections to the data due to off-axis acquisition

In [ ]:
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]),
                               preserve_range=True)

In [ ]:
dp.data.dtype

Align the dataset based on the direct beam position

In [ ]:
dp.center_direct_beam(radius_start=2,
                      radius_finish=5,
                      square_width=10)

In [ ]:
dp.plot(cmap='inferno', vmax=50)

Measure known interplanar spacing to obtain calibration

In [ ]:
dpm = dp.mean((0,1))
dpm.plot(cmap='inferno', vmax=50)
line = pxm.roi.Line2DROI(x1=25.8525, y1=64.5691, x2=120.907, y2=77.0079, linewidth=5.49734)
line.add_widget(dpm)

In [ ]:
trace = line(dpm)
trace.plot()

In [ ]:
recip_d111 = np.sqrt((3/5.6535**2))
recip_cal = recip_d111 / 11.4

Set data calibrations

In [ ]:
dp.set_diffraction_calibration(recip_cal)
dp.set_scan_calibration(10)

Plot the calibrated data

In [ ]:
dp.plot(cmap='inferno', vmax=50)

# <a id='vdf'></a> 3. Virtual Diffraction Imaging & Selecting Regions

## 3.1 Interactive VDF Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [ ]:
roi = pxm.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.07)
dp.plot_interactive_virtual_image(roi=roi, cmap='inferno', vmax=50)

Get the virtual diffraction image associated with the last integration window used interactively

In [ ]:
vdf = dp.get_virtual_image(roi)

Plot the virtual dark-field image

In [ ]:
vdf.plot()

Save the virtual dark-field image as a 32bit tif

In [ ]:
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

## 3.2 Form multiple images using a VDF Generator

Import the VDFGenerator class

In [ ]:
from pyxem.generators.vdf_generator import VDFGenerator

Initialize the VDFGenerator

In [ ]:
vdfgen = VDFGenerator(dp)

Calculate 10 annular VDF images between 0 and 1 reciprocal angstroms

In [ ]:
vdfs = vdfgen.get_concentric_vdf_images(k_min=0,
                                        k_max=1,
                                        k_steps=10)

Plot the VDF images

In [ ]:
vdfs.plot()

Save the stack of VDF image as a 32bit tif stack

In [ ]:
vdfs.change_dtype('float32')
vdfs.save('vdsfeg.tif')

## 3.3 Select a region in the scan

Plot the data with an adjustable marker indicating where to crop the scan region

In [ ]:
reg = pxm.roi.RectangularROI(left=50, top=630, right=290, bottom=870)
dp.plot(cmap='inferno', vmax=50)
reg.add_widget(dp)

Crop the dataset based on the region defined above

In [ ]:
dpc = reg(dp)

Calculate the mean diffraction pattern from the selected region

In [ ]:
dpcm = dpc.mean((0,1))

Plot the mean diffraction pattern from the selected region

In [ ]:
dpcm.plot(cmap='inferno', vmax=50)

# <a id='ml'></a> 4. Unsupervised learning

Perform singular value decomposition (SVD) of the data

In [ ]:
dpc.data = dpc.data.astype('float64')
dpc.decomposition(True, algorithm='svd')

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [ ]:
dpc.plot_explained_variance_ratio()

Plot the decomposition results and have a look at them

In [ ]:
dpc.plot_decomposition_results()

Perform non-negative matrix factorisation (NMF)

In [ ]:
dpc.decomposition(True, algorithm='nmf', output_dimension=3)

Plot the NMF results

In [ ]:
dpc.plot_decomposition_results()

# 5. Azimuthal Integration & Background Subtraction

Obtain an aximuthially integrated diffraction profile and plot it

In [ ]:
rp = dpc.get_radial_profile()

Plot the electron diffraction profile at every probe position

In [ ]:
rp.plot()

Perform a background subtraction

In [ ]:
dpb = dpc.remove_background(method='median', footprint=12)

Plot the background subtracted electron diffraction data

In [ ]:
dpb.plot(cmap='inferno', vmax=50)

# <a id='vec'></a> 6. Peak Finding & Vector Analysis

Interactively tune peak finding parameters

In [ ]:
dpc.find_peaks_interactive()

Perform peak finding on all diffraction patterns in data

In [ ]:
peaks = dpc.find_peaks(method='difference_of_gaussians',
                       min_sigma=1.,
                       max_sigma=6.,
                       sigma_ratio=1.6,
                       threshold=0.04,
                       overlap=0.99)

Check the peaks object type

In [ ]:
peaks

Look at what's in the peaks object

In [ ]:
peaks.data

Plot the found peak positions

In [ ]:
peaks.plot_diffraction_vectors(xlim=0.8, ylim=0.8, distance_threshold=0.01)

Plot found peak positions as an overlay on the data

In [ ]:
peaks.plot_diffraction_vectors_on_signal(dpc, cmap='inferno', vmax=50)

Form a diffracting pixels map to show where peaks were found and plot it

In [ ]:
crystim = peaks.get_diffracting_pixels_map(binary=False)
crystim.plot(cmap='viridis')

Produce a histogram of diffraction vector magnitudes

In [ ]:
bins = np.arange(0, 1.5, recip_cal)
ghist = peaks.get_magnitude_histogram(bins=bins)
ghist.plot()

## 6.1 Imaging using found diffraction conditions

Produce virtual diffraction contrast images for all diffraction vectors

In [ ]:
from pyxem.generators.vdf_generator import VDFGenerator

In [ ]:
vdfgen = VDFGenerator(dpc, peaks)

In [ ]:
vdfs = vdfgen.get_vector_vdf_images(radius=recip_cal*3)

In [ ]:
vdfs.plot()